In [ ]:
!pip install -qU langchain langchain-groq langchain-experimental


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
os.environ["GROQ_API_KEY"] = "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"


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langchain.agents import Tool, initialize_agent

# Initialize the Groq LLM
llm = ChatGroq(
    temperature=0,
    model_name="mixtral-8x7b-32768"  # Replace with your desired model
)


In [ ]:
from langchain.chains import LLMMathChain

# Create the LLMMathChain using the Groq LLM
llm_math = LLMMathChain(llm=llm)

# Initialize the math tool
math_tool = Tool(
    name='Calculator',
    func=llm_math.run,
    description='Useful for when you need to answer questions about math.'
)


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm_math/base.py:56: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(


In [ ]:
from langchain.prompts import PromptTemplate

# Create a general-purpose LLM tool
prompt = PromptTemplate(
    input_variables=["query"],
    template="{query}"
)

llm_chain = LLMChain(llm=llm, prompt=prompt)

# Initialize the LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm_chain.run,
    description='Use this tool for general purpose queries and logic'
)

# Combine the tools
tools = [math_tool, llm_tool]


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [ ]:
from langchain.agents import initialize_agent

# Initialize the zero-shot agent with the tools
zero_shot_agent = initialize_agent(
    agent="zero-shot-react-description",
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [ ]:
zero_shot_agent("9.9 is greater or 9.11 is greater")




> Entering new AgentExecutor chain...
The question is asking which of the two numbers, 9.9 or 9.11, is greater. This is a comparison question that involves numbers, so I can use the Calculator tool to determine the answer.

Action: Calculator
Action Input: 9.9 > 9.11
Observation: Answer: True
Thought:The calculator has returned True, which means 9.9 is indeed greater than 9.11.
Final Answer: 9.9 is greater than 9.11.

> Finished chain.


{'input': '9.9 is greater or 9.11 is greater',
 'output': '9.9 is greater than 9.11.'}

In [ ]:
tools.append(llm_tool)

In [ ]:
zero_shot_agent("what is the capital of Norway?")



> Entering new AgentExecutor chain...
I don't have this information readily available, I will need to use the Language Model tool to find the answer.
Action: Language Model
Action Input: What is the capital of Norway?
Observation: The capital of Norway is Oslo. Established around 1000 AD, Oslo is the oldest of the Scandinavian capitals. Modern Oslo is a cosmopolitan city with a diverse population and a rich cultural scene, home to numerous museums, galleries, and theaters, as well as a vibrant nightlife. It is also known for its beautiful parks and gardens, and its proximity to nature, with forests, mountains, and fjords all within easy reach of the city center.
Thought:I now know the final answer
Final Answer: The capital of Norway is Oslo.

> Finished chain.


{'input': 'what is the capital of Norway?',
 'output': 'The capital of Norway is Oslo.'}

In [ ]:
zero_shot_agent("What is (4.5*2.1)^2.2?")




> Entering new AgentExecutor chain...
To answer this question, I need to perform a series of mathematical operations. I'll first multiply 4.5 and 2.1, then raise the result to the power of 2.2. I can use the Calculator tool for this.

Action: Calculator
Action Input: (4.5*2.1)**2.2

Observation: Answer: 139.94261298333066
Thought:I now know the final answer.
Final Answer: The value of (4.5*2.1)^2.2 is approximately 139.94.

> Finished chain.


{'input': 'What is (4.5*2.1)^2.2?',
 'output': 'The value of (4.5*2.1)^2.2 is approximately 139.94.'}

In [ ]:
# Import necessary libraries for the CSV agent
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

# Create the CSV agent
csv_agent = create_csv_agent(
    llm,  # Using the Groq LLM we initialized earlier
    "/content/insurance.csv",  # Replace with the path to your CSV file
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True  # Add this line to enable the feature
)

# Example query to interact with the CSV
response = csv_agent("How many females having bmi less than 30?")
print(response)



> Entering new AgentExecutor chain...
Thought: To answer this question, I need to filter the dataframe for rows where sex is 'female' and bmi is less than 30. Then, I can count the number of rows that meet these conditions.

Action: python_repl_ast
Action Input: len(df[(df['sex'] == 'female') & (df['bmi'] < 30)])328I now know the final answer.
Final Answer: There are 328 females having a BMI less than 30.

> Finished chain.
{'input': 'How many females having bmi less than 30?', 'output': 'There are 328 females having a BMI less than 30.'}


---